In [191]:
import pandas as pd
import numpy as np

## Importing

In [192]:
df2 = pd.read_csv('motor-vehicle-crashes-case-information-three-year-window.csv') 

In [193]:
df = pd.read_csv('motor-vehicle-crashes-case-information-three-year-window.csv') 

In [194]:
counties = pd.read_csv('counties.csv')

In [195]:
df.head()

,Year,Crash Descriptor,Time,Date,Day of Week,Police Report,Lighting Conditions,Municipality,Collision Type Descriptor,County Name,Road Descriptor,Weather Conditions,Traffic Control Device,Road Surface Conditions,DOT Reference Marker Location,Pedestrian Bicyclist Action,Event Descriptor,Number of Vehicles Involved
0,2016,Property Damage Accident,11:30,2016-12-31T00:00:00.000,Saturday,Y,Daylight,QUEENS,RIGHT TURN (5),QUEENS,Straight and Level,Clear,Traffic Signal,Dry,NaN,Not Applicable,"Other Motor Vehicle, Collision With",2
1,2016,Injury Accident,18:39,2016-12-31T00:00:00.000,Saturday,Y,Dark-Road Lighted,NEW YORK,OTHER,NEW YORK,Straight and Level,Clear,Traffic Signal,Dry,NaN,Unknown,"Bicyclist, Collision With",1
2,2016,Property Damage Accident,14:45,2016-12-31T00:00:00.000,Saturday,Y,Daylight,ITHACA,OTHER,TOMPKINS,Curve and Grade,Cloudy,None,Dry,13 36033003,Not Applicable,"Guide Rail - Not At End, Collision With Fixed ...",1
3,2016,Property Damage Accident,12:32,2016-12-31T00:00:00.000,Saturday,Y,Daylight,COLDEN,OTHER,ERIE,Straight and Level,Snow,None,Snow/Ice,NaN,Not Applicable,"Snow Embankment, Collision With Fixed Object",1
4,2016,Property Damage & Injury Accident,15:25,2016-12-31T00:00:00.000,Saturday,Y,Daylight,NEW YORK,REAR END,NEW YORK,Straight and Level,Clear,Traffic Signal,Dry,NaN,Not Applicable,"Other Motor Vehicle, Collision With",2


In [196]:
df.shape

(895916, 18)

## Pruning 

### Municipality VS County
Figuring out which is better and which to drop

In [197]:
df['Municipality'].value_counts()

QUEENS                  59884
KINGS                   58586
HEMPSTEAD               41149
NEW YORK                35805
BRONX                   32424
                        ...  
COVE NECK                   2
SOUTH BLOOMING GROVE        1
AMES                        1
SALTAIRE                    1
OCEAN BEACH                 1
Name: Municipality, Length: 1293, dtype: int64

In [198]:
df['County Name'].value_counts()

NASSAU       98983
SUFFOLK      96013
QUEENS       59884
KINGS        58586
ERIE         51699
             ...  
SCHOHARIE     1982
YATES         1625
LEWIS         1511
SCHUYLER      1278
HAMILTON       630
Name: County Name, Length: 63, dtype: int64

Municipality is too long and has to go

In [199]:
df = df.drop(['Municipality'] ,axis=1)

### DOT Reference Marker

<img src="marker.jpeg">

**Top row:** This is the route designation. The first three spaces carry the numerical portion of the route designation (one to three digits) and the last space carries the alphabetic suffix, if any. If the route number is only one digit, this digit is placed in the third space from the left; a two-digit number starts in the second space. Thus the numerals are always adjacent to the alphabetic suffix.

The route designation given on markers is often different from the actual touring or reference route number. Sometimes a route has been absorbed by another route, or has been relocated on a new alignment and a new route designtaion placed on the old alignment. Also, a touring route may have been decommissioned and parts of it reassigned as reference routes. In many of these cases, reference markers show the older route numbers because the markers are usually only replaced as part of a major maintenance or construction project.

**Middle row:** The first digit gives the Region number. NYSDOT divides the State into 11 Regions, which are jurisdictional divisions. For Regions 1 through 9, the appropriate numeral is used. For Region 10, the numeral 0 is used, and for Region 11 the letter X is used.

The second digit is the county code. The counties are arranged alphabetically within their respective Regions and assigned numbers according to this listing. The first two digits combined therefore give a unique code to each of the 62 counties.

The third and fourth digits give the county order number. This number begins with 01 at the south or west terminus of the route and increases by one each time a county line is crossed. If a route enters and leaves a county more than once, each crossing is counted. [In Region 11 (New York City), this number is expressed only by the last digit in the middle row. The third digit will then be M for mainlines and C for collector-distributor roads.]

**Bottom row:** The first digit is the control segment number. A control segment is a division of a route within a county, and is almost always measured by city lines. At each county line, the control segment number is reset to 1 and increases with each city line crossing (again, repeated crossings are counted separately). In the case of the three cities (Saratoga Springs, Rome and Oneida) that have high and low tax boundaries, the control segment theoretically changes only at the low tax (outer) boundary. On existing markers, however, this digit changes only at the high tax (inner) boundary.

In [200]:
df['DOT Reference Marker Location'].value_counts()

495 03021043    248
95IX1M22008     148
495 03021106    142
9 82052016      137
907W87024001    132
               ... 
5 44052219        1
268 93011080      1
38 31041179       1
21 44053058       1
9 81061042        1
Name: DOT Reference Marker Location, Length: 80299, dtype: int64

In [201]:
df['DOT Reference Marker Location'].count().sum()

313582

In [202]:
df['DOT Reference Marker Location'].isnull().sum()

582334

This is categorical data disguised as digits and there's too many nulls to make this worth it.

In [203]:
df = df.drop(['DOT Reference Marker Location'] ,axis=1)

### Weather Conditions vs Road Surface Conditions
Here is an interesting one, we have two kinds of weather related features, one for the environment and one for the road. Let's see if we can drop one. 

#### Domain Knowledge 
According to the Federal Highway Administration, 23 percent of all accidents are weather-related. Weather-related crashes include crashes that take place during rain, sleet, snow, fog, severe crosswinds, and/or blowing snow, sand or debris.

Of the 1.3 million plus weather-related crashes each year, less than half involve a winter weather condition. While 74 percent of weather-related accidents occur on wet pavement (with 46 percent of weather-related accidents taking place while it is raining), far less take place during ice, sleet and snow conditions. In fact, of all the weather-related crashes, only 17 percent happen while it is snowing or sleeting, 12 percent occur on icy pavement, and 14 percent take place on snowy or slushy pavement. Some accidents may involve a combination of weather conditions, such as rainfall and wet pavement or a mixture of ice and snow.

In [204]:
df['Weather Conditions'].value_counts()

Clear                       502596
Cloudy                      192106
Rain                         74102
Unknown                      67521
Snow                         48262
Sleet/Hail/Freezing Rain      7886
Fog/Smog/Smoke                2847
Other*                         596
Name: Weather Conditions, dtype: int64

In [205]:
df['Weather Conditions'].isnull().sum()

0

In [206]:
df['Road Surface Conditions'].value_counts()

Dry              616638
Wet              129290
Snow/Ice          76546
Unknown           66661
Slush              4846
Other              1140
Muddy               506
Flooded Water       289
Name: Road Surface Conditions, dtype: int64

In [207]:
df['Road Surface Conditions'].isnull().sum()

0

I might be able to get away with turning these both into 1/0 columns based on if the road is dry or not and if the weather is clear or not. I'm going to end up combining 'Clear' and 'Cloudy' let the road condition catch any difference. The "Unkown" entries really bother me though, I'm going to make the assumption that they're not clear for this case. 

### Police Report?
How useful or complete is this? It might already be set up for a 1/0 method so no point in getting ride of it

In [208]:
df['Police Report'].value_counts()

Y    766631
N    129285
Name: Police Report, dtype: int64

In [209]:
df['Police Report'].isnull().sum()

0

My thoughts on this are that of course a majority of them are police report related, afterall that is how one way data is collected. The ones with no report warrent a second look.

In [210]:
no_report = df.loc[(df['Police Report'] == 'N')]

In [211]:
no_report['Crash Descriptor'].value_counts()

Property Damage Accident             99057
Injury Accident                      20967
Property Damage & Injury Accident     9261
Name: Crash Descriptor, dtype: int64

I'm going to keep this for now and turn it into a 1/0. It contains zero fatalaties which means it can be helpful.

### Pedestrian Bicyclist Action?
What does this even look like?

In [212]:
df['Pedestrian Bicyclist Action'].value_counts()

Not Applicable                                          835128
Crossing, With Signal                                    13857
Crossing, No Signal or Crosswalk                         10216
Unknown                                                   9134
Riding/Walking/Skating Along Highway With Traffic         5762
Crossing, Against Signal                                  5102
Crossing, No Signal, Marked Crosswalk                     4232
Other Actions in Roadway                                  4153
Not in Roadway (Indicate)                                 2650
Riding/Walking/Skating Along Highway Against Traffic      1996
Emerging from in Front of/Behind Parked Vehicle           1888
Working in Roadway                                         711
Getting On/Off Vehicle Other than School Bus               587
Playing in Roadway                                         418
Going to/From Stopped School Bus                            68
Pushing/Working On Car                                 

Other features already give us this kind of information.

In [213]:
df = df.drop(['Pedestrian Bicyclist Action'] ,axis=1)

### Collision Type Descriptor 

In [214]:
df['Collision Type Descriptor'].value_counts()

OTHER             357120
REAR END          194002
RIGHT ANGLE       100063
OVERTAKING         87607
Unknown            70769
LEFT TURN (3)      30834
SIDESWIPE          15797
LEFT TURN (0)      14089
HEAD ON            11034
RIGHT TURN (6)      7539
RIGHT TURN (5)      7062
Name: Collision Type Descriptor, dtype: int64

The amount of "Other" and "Unknown" in this feature makes it not worth having around.

In [215]:
df = df.drop(['Collision Type Descriptor'] ,axis=1)

In [216]:
df.head()

,Year,Crash Descriptor,Time,Date,Day of Week,Police Report,Lighting Conditions,County Name,Road Descriptor,Weather Conditions,Traffic Control Device,Road Surface Conditions,Event Descriptor,Number of Vehicles Involved
0,2016,Property Damage Accident,11:30,2016-12-31T00:00:00.000,Saturday,Y,Daylight,QUEENS,Straight and Level,Clear,Traffic Signal,Dry,"Other Motor Vehicle, Collision With",2
1,2016,Injury Accident,18:39,2016-12-31T00:00:00.000,Saturday,Y,Dark-Road Lighted,NEW YORK,Straight and Level,Clear,Traffic Signal,Dry,"Bicyclist, Collision With",1
2,2016,Property Damage Accident,14:45,2016-12-31T00:00:00.000,Saturday,Y,Daylight,TOMPKINS,Curve and Grade,Cloudy,None,Dry,"Guide Rail - Not At End, Collision With Fixed ...",1
3,2016,Property Damage Accident,12:32,2016-12-31T00:00:00.000,Saturday,Y,Daylight,ERIE,Straight and Level,Snow,None,Snow/Ice,"Snow Embankment, Collision With Fixed Object",1
4,2016,Property Damage & Injury Accident,15:25,2016-12-31T00:00:00.000,Saturday,Y,Daylight,NEW YORK,Straight and Level,Clear,Traffic Signal,Dry,"Other Motor Vehicle, Collision With",2


### Lighting Conditons

In [217]:
df['Lighting Conditions'].value_counts()

Daylight               547660
Dark-Road Lighted      149542
Dark-Road Unlighted     86677
Unknown                 67765
Dusk                    26107
Dawn                    18165
Name: Lighting Conditions, dtype: int64

Keep this around and convert it into a new column or two for whether it's daylight or not. I feel comfortable combining everything else into that vs daylight. 

### Year
We really don't need year to be included in this

In [218]:
df = df.drop(['Year'] ,axis=1)

## Engineering

### Time = just hour

In [219]:
def hour_split(time):
    split = time.split(':')
    return int(split[0])

In [220]:
df['Time'] = df.Time.apply(hour_split)

### Day of Week Number Conversion

In [221]:
df['Day of Week'].value_counts()

Friday       147681
Thursday     135332
Wednesday    132954
Tuesday      131951
Monday       128593
Saturday     118697
Sunday       100708
Name: Day of Week, dtype: int64

In [222]:
def day_week(day):
    days_of_week = {'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4,
                   'Friday':5, 'Saturday':6, 'Sunday':7}
    
    return days_of_week[day]       

In [223]:
df['Day of Week'] = df['Day of Week'].apply(day_week)

### Police Report

In [224]:
def report_convert(report):
    if report == 'Y':
        return 1
    else:
        return 0

In [225]:
df['Police Report'] = df['Police Report'].apply(report_convert)

### Date into Month

In [226]:
def month_ripper(date):
    split = date.split('-')
    return int(split[1])

In [227]:
df['Date'] = df['Date'].apply(month_ripper)

In [228]:
df.rename(columns={'Date': 'Month'}, inplace=True)

### Daylight/Lit Road/Twilight Check

In [229]:
df['Lit Dark Road?'] = df['Lighting Conditions']

In [230]:
df['Twilight Hour?'] = df2['Lighting Conditions']

In [231]:
def day_light(condition):
    if condition == 'Daylight':
        return 1
    else:
        return 0

In [232]:
def lit_road_check(condition):
    if condition == 'Dark-Road Lighted':
        return 1
    else:
        return 0

In [233]:
def twilight_check(condition):
    if condition == 'Dusk':
        return 1
    elif condition == 'Dawn':
        return 1
    else:
        return 0

In [234]:
df['Lighting Conditions'].value_counts()

Daylight               547660
Dark-Road Lighted      149542
Dark-Road Unlighted     86677
Unknown                 67765
Dusk                    26107
Dawn                    18165
Name: Lighting Conditions, dtype: int64

In [235]:
df['Lighting Conditions'] = df['Lighting Conditions'].apply(day_light)

In [236]:
df['Lit Dark Road?'] = df['Lit Dark Road?'].apply(lit_road_check)

In [237]:
df['Twilight Hour?'] = df['Twilight Hour?'].apply(twilight_check)

In [238]:
twilight_check('Daylight')

0

In [239]:
df.rename(columns={'Lighting Conditions': 'Daylight?'}, inplace=True)

### Traffic Control Device

In [240]:
df['Traffic Control Device'].value_counts()

None                                      474034
Traffic Signal                            187597
Unknown                                    80073
Stop Sign                                  74412
No Passing Zone                            54433
Other                                       8194
Yield Sign                                  6986
Construction Work Area                      3238
Flashing Light                              2110
Not Applicable                              1040
RR Crossing Gates                            844
Police/Fire Emergency                        671
Stopped School Bus-Red Lights Flashing       575
Officer/Guard                                497
Maintenance Work Area                        394
School Zone                                  374
RR Crossing Sign                             210
Utility Work Area                            150
RR Crossing Flashing Light                    84
Name: Traffic Control Device, dtype: int64

In [241]:
def control_check(device):
    if device in ['None', 'Not Applicable']:
        return 0
    else:
        return 1

In [242]:
df['Traffic Control Device'] = df['Traffic Control Device'].apply(control_check)

In the future I want to boil down these traffic control devices and figure out which ones are best correlated with my target.

### Road Type

In [243]:
df['Road Descriptor'].value_counts()

Straight and Level        627342
Straight and Grade         93101
Unknown                    67083
Curve and Level            49453
Curve and Grade            40284
Straight at Hill Crest     13978
Curve at Hill Crest         4675
Name: Road Descriptor, dtype: int64

**Halfway through I found this online, I forgot that if you convert a bool statement into an into it will be a 1/0**

In [244]:
df['Straight Road?'] = df['Road Descriptor'].str.contains('Straight').astype(int)

In [245]:
df['Level Road?'] = df['Road Descriptor'].str.contains('Level').astype(int)

In [246]:
df['Hill Crest?'] = df['Road Descriptor'].str.contains('Hill Crest').astype(int)

In [247]:
df = df.drop(['Road Descriptor'] ,axis=1)

### Road Surface Condition

In [248]:
df['Road Surface Conditions'].value_counts()

Dry              616638
Wet              129290
Snow/Ice          76546
Unknown           66661
Slush              4846
Other              1140
Muddy               506
Flooded Water       289
Name: Road Surface Conditions, dtype: int64

In [249]:
df['Road Surface Conditions'] = df['Road Surface Conditions'].str.contains('Dry').astype(int)

In [250]:
df['Dry Road?'] = df['Road Surface Conditions']

In [251]:
df.drop(['Road Surface Conditions'] ,axis=1, inplace=True)

### Weather Conditions

In [252]:
df.head()

,Crash Descriptor,Time,Month,Day of Week,Police Report,Daylight?,County Name,Weather Conditions,Traffic Control Device,Event Descriptor,Number of Vehicles Involved,Lit Dark Road?,Twilight Hour?,Straight Road?,Level Road?,Hill Crest?,Dry Road?
0,Property Damage Accident,11,12,6,1,1,QUEENS,Clear,1,"Other Motor Vehicle, Collision With",2,0,0,1,1,0,1
1,Injury Accident,18,12,6,1,0,NEW YORK,Clear,1,"Bicyclist, Collision With",1,1,0,1,1,0,1
2,Property Damage Accident,14,12,6,1,1,TOMPKINS,Cloudy,0,"Guide Rail - Not At End, Collision With Fixed ...",1,0,0,0,0,0,1
3,Property Damage Accident,12,12,6,1,1,ERIE,Snow,0,"Snow Embankment, Collision With Fixed Object",1,0,0,1,1,0,0
4,Property Damage & Injury Accident,15,12,6,1,1,NEW YORK,Clear,1,"Other Motor Vehicle, Collision With",2,0,0,1,1,0,1


In [253]:
def weather_convertor(weather):
    '''
    Checks to see if the weather was clear/cloudy and returns a 1, all others get 0.
    '''
    return int(weather in ['Clear','Cloudy'])

In [254]:
df['Weather Conditions'] = df['Weather Conditions'].apply(weather_convertor)

In [255]:
df.rename(columns={'Weather Conditions': 'Precipitation?'}, inplace=True)

### County DUI Rates
Bringing in high DUI rate counties

In [256]:
counties_dui = pd.read_csv('county_dui_rate.csv')

In [257]:
counties_dui.head()

,County,Rate
0,Albany County,"1,247"
1,Allegany County,941
2,Bronx County,"7,091"
3,Broome County,"1,278"
4,Cattaraugus County,"1,015"


In [258]:
counties_dui['Rate'] = counties_dui['Rate'].str.replace(',', '')
counties_dui['Rate'] = counties_dui['Rate'].astype(float)
counties_dui['Rate'] = counties_dui['Rate'].astype(int)
dui_dict = counties_dui.set_index('County').to_dict()

In [259]:
dui_dict

{'Rate': {'Albany County': 1247,
  'Allegany County': 941,
  'Bronx County': 7091,
  'Broome County': 1278,
  'Cattaraugus County': 1015,
  'Cayuga County': 1276,
  'Chautauqua County': 1040,
  'Chemung County': 1264,
  'Chenango County': 1151,
  'Clinton County': 1341,
  'Columbia County': 1239,
  'Cortland County': 952,
  'Delaware County': 883,
  'Dutchess County': 1381,
  'Erie County': 1438,
  'Essex County': 878,
  'Franklin County': 1042,
  'Fulton County': 1027,
  'Genesee County': 818,
  'Greene County': 858,
  'Hamilton County': 552,
  'Herkimer County': 1157,
  'Jefferson County': 1017,
  'Kings County': 6448,
  'Lewis County': 1252,
  'Livingston County': 1258,
  'Madison County': 1419,
  'Monroe County': 1354,
  'Montgomery County': 1070,
  'Nassau County': 2013,
  'New York County': 6594,
  'Niagara County': 1292,
  'Oneida County': 1369,
  'Onondaga County': 1363,
  'Ontario County': 1307,
  'Orange County': 1254,
  'Orleans County': 1223,
  'Oswego County': 1301,
  'Ots

In [260]:
'Hello' + " County"

'Hello County'

In [261]:
def dui_convert(county):
    '''
    Converts the county into the suitable format and then checks it against the dictionary of counties,
    returning its paired population density
    '''
    capital_case = county.title() + " County"
    if capital_case == 'Unknown County':
        return np.nan
    else:
        return dui_dict['Rate'][capital_case]

In [262]:
df['Pop per DUI'] = df['County Name'].apply(dui_convert)

In [263]:
df.dropna(inplace=True)

In [264]:
df['Pop per DUI'] = df['Pop per DUI'].astype(int)

In [265]:
df.dropna(inplace=True)

In [266]:
df.head()

,Crash Descriptor,Time,Month,Day of Week,Police Report,Daylight?,County Name,Precipitation?,Traffic Control Device,Event Descriptor,Number of Vehicles Involved,Lit Dark Road?,Twilight Hour?,Straight Road?,Level Road?,Hill Crest?,Dry Road?,Pop per DUI
0,Property Damage Accident,11,12,6,1,1,QUEENS,1,1,"Other Motor Vehicle, Collision With",2,0,0,1,1,0,1,5579
1,Injury Accident,18,12,6,1,0,NEW YORK,1,1,"Bicyclist, Collision With",1,1,0,1,1,0,1,6594
2,Property Damage Accident,14,12,6,1,1,TOMPKINS,1,0,"Guide Rail - Not At End, Collision With Fixed ...",1,0,0,0,0,0,1,1261
3,Property Damage Accident,12,12,6,1,1,ERIE,0,0,"Snow Embankment, Collision With Fixed Object",1,0,0,1,1,0,0,1438
4,Property Damage & Injury Accident,15,12,6,1,1,NEW YORK,1,1,"Other Motor Vehicle, Collision With",2,0,0,1,1,0,1,6594


### County Density
I decided to bring in outside data related to how densly populated the county is

In [267]:
counties.head()

,County,Population_Density
0,New York City,"27,011.84"
1,Bronx,"32,900.43"
2,Kings,"35,367.13"
3,New York,"69,464.43"
4,Queens,"20,553.97"


In [268]:
counties['Population_Density'] = counties['Population_Density'].str.replace(',', '')

In [269]:
counties['Population_Density'] = counties['Population_Density'].astype(float)

In [270]:
counties['Population_Density'] = counties['Population_Density'].astype(int)

In [271]:
df['County Name'] = df['County Name'].str.replace('.','')

In [272]:
county_dict = counties.set_index('County').to_dict()

In [273]:
def pop_convert(county):
    '''
    Converts the county into the suitable format and then checks it against the dictionary of counties,
    returning its paired population density
    '''
    capital_case = county.title() 
    if capital_case == 'Unknown':
        return np.nan
    else:
        return county_dict['Population_Density'][capital_case]

In [274]:
df['Population Density'] = df['County Name'].apply(pop_convert)

In [275]:
df['Population Density'] = df['Population Density'].astype(int)

In [276]:
df.dropna(inplace=True)

In [277]:
df.head()

,Crash Descriptor,Time,Month,Day of Week,Police Report,Daylight?,County Name,Precipitation?,Traffic Control Device,Event Descriptor,Number of Vehicles Involved,Lit Dark Road?,Twilight Hour?,Straight Road?,Level Road?,Hill Crest?,Dry Road?,Pop per DUI,Population Density
0,Property Damage Accident,11,12,6,1,1,QUEENS,1,1,"Other Motor Vehicle, Collision With",2,0,0,1,1,0,1,5579,20553
1,Injury Accident,18,12,6,1,0,NEW YORK,1,1,"Bicyclist, Collision With",1,1,0,1,1,0,1,6594,69464
2,Property Damage Accident,14,12,6,1,1,TOMPKINS,1,0,"Guide Rail - Not At End, Collision With Fixed ...",1,0,0,0,0,0,1,1261,213
3,Property Damage Accident,12,12,6,1,1,ERIE,0,0,"Snow Embankment, Collision With Fixed Object",1,0,0,1,1,0,0,1438,881
4,Property Damage & Injury Accident,15,12,6,1,1,NEW YORK,1,1,"Other Motor Vehicle, Collision With",2,0,0,1,1,0,1,6594,69464


### Tackling Event Descriptor 
We need to pair these down to causes and participants 

In [278]:
df['Event Descriptor'].value_counts()

Other Motor Vehicle, Collision With                             607981
Deer                                                             67871
Pedestrian, Collision With                                       41041
Guide Rail - Not At End, Collision With Fixed Object             24158
Earth Embankment/Rock Cut/Ditch, Collision With Fixed Object     20615
Light Support/Utility Pole, Collision With Fixed Object          18066
Bicyclist, Collision With                                        17294
Tree, Collision With Fixed Object                                17199
Other Fixed Object*, Collision With Fixed Object                  9372
Animal, Collision With                                            8486
Other Object (Not Fixed)*, Collision With                         7997
Sign Post, Collision With Fixed Object                            7710
Building/Wall, Collision With Fixed Object                        5245
Other*, Non-Collision                                             5160
Curbin

In [279]:
df['Collision?'] = df['Event Descriptor'].str.contains('Collision'or'Deer').astype(int)
df['Pedestrian'] = df['Event Descriptor'].str.contains('Pedestrian'or'Skater').astype(int)
df['Other Vehicle?'] = df['Event Descriptor'].str.contains('Vehicle').astype(int)
df['Cyclist?'] = df['Event Descriptor'].str.contains('Bicyclist').astype(int)
df['Fixed Object?'] = df['Event Descriptor'].str.contains('Fixed Object').astype(int)
df['Unkown Cause'] = df['Event Descriptor'].str.contains('Unknown').astype(int)
df['Explosion/Fire'] = df['Event Descriptor'].str.contains('Fire/Explosion').astype(int)
df['Animal Involved?'] = df['Event Descriptor'].str.contains('Animal'or'Deer').astype(int)
df['Overturned?'] = df['Event Descriptor'].str.contains('Overturned').astype(int)
df['Train?'] = df['Event Descriptor'].str.contains('Train').astype(int)
df['Submerged Car?'] = df['Event Descriptor'].str.contains('Submersion').astype(int)

In [280]:
df.head()

,Crash Descriptor,Time,Month,Day of Week,Police Report,Daylight?,County Name,Precipitation?,Traffic Control Device,Event Descriptor,...,Pedestrian,Other Vehicle?,Cyclist?,Fixed Object?,Unkown Cause,Explosion/Fire,Animal Involved?,Overturned?,Train?,Submerged Car?
0,Property Damage Accident,11,12,6,1,1,QUEENS,1,1,"Other Motor Vehicle, Collision With",...,0,1,0,0,0,0,0,0,0,0
1,Injury Accident,18,12,6,1,0,NEW YORK,1,1,"Bicyclist, Collision With",...,0,0,1,0,0,0,0,0,0,0
2,Property Damage Accident,14,12,6,1,1,TOMPKINS,1,0,"Guide Rail - Not At End, Collision With Fixed ...",...,0,0,0,1,0,0,0,0,0,0
3,Property Damage Accident,12,12,6,1,1,ERIE,0,0,"Snow Embankment, Collision With Fixed Object",...,0,0,0,1,0,0,0,0,0,0
4,Property Damage & Injury Accident,15,12,6,1,1,NEW YORK,1,1,"Other Motor Vehicle, Collision With",...,0,1,0,0,0,0,0,0,0,0


In [281]:
df.drop(['Event Descriptor'] ,axis=1, inplace=True)

### County Dummies
Final Step!

In [282]:
df = pd.get_dummies(df,columns=['County Name'])

In [283]:
df.head()

,Crash Descriptor,Time,Month,Day of Week,Police Report,Daylight?,Precipitation?,Traffic Control Device,Number of Vehicles Involved,Lit Dark Road?,...,County Name_SULLIVAN,County Name_TIOGA,County Name_TOMPKINS,County Name_ULSTER,County Name_WARREN,County Name_WASHINGTON,County Name_WAYNE,County Name_WESTCHESTER,County Name_WYOMING,County Name_YATES
0,Property Damage Accident,11,12,6,1,1,1,1,2,0,...,0,0,0,0,0,0,0,0,0,0
1,Injury Accident,18,12,6,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,Property Damage Accident,14,12,6,1,1,1,0,1,0,...,0,0,1,0,0,0,0,0,0,0
3,Property Damage Accident,12,12,6,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Property Damage & Injury Accident,15,12,6,1,1,1,1,2,0,...,0,0,0,0,0,0,0,0,0,0


## Fix Columns

In [284]:
cols = list(df.columns.values)

In [285]:
cols

['Crash Descriptor',
 'Time',
 'Month',
 'Day of Week',
 'Police Report',
 'Daylight?',
 'Precipitation?',
 'Traffic Control Device',
 'Number of Vehicles Involved',
 'Lit Dark Road?',
 'Twilight Hour?',
 'Straight Road?',
 'Level Road?',
 'Hill Crest?',
 'Dry Road?',
 'Pop per DUI',
 'Population Density',
 'Collision?',
 'Pedestrian',
 'Other Vehicle?',
 'Cyclist?',
 'Fixed Object?',
 'Unkown Cause',
 'Explosion/Fire',
 'Animal Involved?',
 'Overturned?',
 'Train?',
 'Submerged Car?',
 'County Name_ALBANY',
 'County Name_ALLEGANY',
 'County Name_BRONX',
 'County Name_BROOME',
 'County Name_CATTARAUGUS',
 'County Name_CAYUGA',
 'County Name_CHAUTAUQUA',
 'County Name_CHEMUNG',
 'County Name_CHENANGO',
 'County Name_CLINTON',
 'County Name_COLUMBIA',
 'County Name_CORTLAND',
 'County Name_DELAWARE',
 'County Name_DUTCHESS',
 'County Name_ERIE',
 'County Name_ESSEX',
 'County Name_FRANKLIN',
 'County Name_FULTON',
 'County Name_GENESEE',
 'County Name_GREENE',
 'County Name_HAMILTON',
 '

In [286]:
df.head()

,Crash Descriptor,Time,Month,Day of Week,Police Report,Daylight?,Precipitation?,Traffic Control Device,Number of Vehicles Involved,Lit Dark Road?,...,County Name_SULLIVAN,County Name_TIOGA,County Name_TOMPKINS,County Name_ULSTER,County Name_WARREN,County Name_WASHINGTON,County Name_WAYNE,County Name_WESTCHESTER,County Name_WYOMING,County Name_YATES
0,Property Damage Accident,11,12,6,1,1,1,1,2,0,...,0,0,0,0,0,0,0,0,0,0
1,Injury Accident,18,12,6,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,Property Damage Accident,14,12,6,1,1,1,0,1,0,...,0,0,1,0,0,0,0,0,0,0
3,Property Damage Accident,12,12,6,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Property Damage & Injury Accident,15,12,6,1,1,1,1,2,0,...,0,0,0,0,0,0,0,0,0,0


In [287]:
df = df[['Crash Descriptor',
 'Police Report',
 'Traffic Control Device',
 'Number of Vehicles Involved',
 'Population Density',
 'Pop per DUI',
 'Time',
 'Month',
 'Day of Week',
 'Precipitation?',
 'Daylight?',
 'Lit Dark Road?',
 'Twilight Hour?',
 'Straight Road?',
 'Level Road?',
 'Hill Crest?',
 'Dry Road?',
 'Collision?',
 'Pedestrian',
 'Other Vehicle?',
 'Cyclist?',
 'Fixed Object?',
 'Unkown Cause',
 'Explosion/Fire',
 'Animal Involved?',
 'Overturned?',
 'Train?',
 'Submerged Car?',
 'County Name_ALBANY',
 'County Name_ALLEGANY',
 'County Name_BRONX',
 'County Name_BROOME',
 'County Name_CATTARAUGUS',
 'County Name_CAYUGA',
 'County Name_CHAUTAUQUA',
 'County Name_CHEMUNG',
 'County Name_CHENANGO',
 'County Name_CLINTON',
 'County Name_COLUMBIA',
 'County Name_CORTLAND',
 'County Name_DELAWARE',
 'County Name_DUTCHESS',
 'County Name_ERIE',
 'County Name_ESSEX',
 'County Name_FRANKLIN',
 'County Name_FULTON',
 'County Name_GENESEE',
 'County Name_GREENE',
 'County Name_HAMILTON',
 'County Name_HERKIMER',
 'County Name_JEFFERSON',
 'County Name_KINGS',
 'County Name_LEWIS',
 'County Name_LIVINGSTON',
 'County Name_MADISON',
 'County Name_MONROE',
 'County Name_MONTGOMERY',
 'County Name_NASSAU',
 'County Name_NEW YORK',
 'County Name_NIAGARA',
 'County Name_ONEIDA',
 'County Name_ONONDAGA',
 'County Name_ONTARIO',
 'County Name_ORANGE',
 'County Name_ORLEANS',
 'County Name_OSWEGO',
 'County Name_OTSEGO',
 'County Name_PUTNAM',
 'County Name_QUEENS',
 'County Name_RENSSELAER',
 'County Name_RICHMOND',
 'County Name_ROCKLAND',
 'County Name_SARATOGA',
 'County Name_SCHENECTADY',
 'County Name_SCHOHARIE',
 'County Name_SCHUYLER',
 'County Name_SENECA',
 'County Name_ST LAWRENCE',
 'County Name_STEUBEN',
 'County Name_SUFFOLK',
 'County Name_SULLIVAN',
 'County Name_TIOGA',
 'County Name_TOMPKINS',
 'County Name_ULSTER',
 'County Name_WARREN',
 'County Name_WASHINGTON',
 'County Name_WAYNE',
 'County Name_WESTCHESTER',
 'County Name_WYOMING',
 'County Name_YATES']]


## Save!

In [289]:
df.head()

,Crash Descriptor,Police Report,Traffic Control Device,Number of Vehicles Involved,Population Density,Pop per DUI,Time,Month,Day of Week,Precipitation?,...,County Name_SULLIVAN,County Name_TIOGA,County Name_TOMPKINS,County Name_ULSTER,County Name_WARREN,County Name_WASHINGTON,County Name_WAYNE,County Name_WESTCHESTER,County Name_WYOMING,County Name_YATES
0,Property Damage Accident,1,1,2,20553,5579,11,12,6,1,...,0,0,0,0,0,0,0,0,0,0
1,Injury Accident,1,1,1,69464,6594,18,12,6,1,...,0,0,0,0,0,0,0,0,0,0
2,Property Damage Accident,1,0,1,213,1261,14,12,6,1,...,0,0,1,0,0,0,0,0,0,0
3,Property Damage Accident,1,0,1,881,1438,12,12,6,0,...,0,0,0,0,0,0,0,0,0,0
4,Property Damage & Injury Accident,1,1,2,69464,6594,15,12,6,1,...,0,0,0,0,0,0,0,0,0,0


In [288]:
df.to_csv('crash_fixed.csv')